# Data Preparation

This parts aim to prepare the data before exploring it and building a prediction model. You must execute notebook `data_retrieval.ipynb` before executing this one.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List
import random

random.seed(42)

In [2]:
games = pd.read_csv('data/game_scores.csv', parse_dates=['date'])
stats = pd.read_csv('data/games_statistics.csv')
raw_data = games.merge(stats, on='boxscore_url', how='inner')
raw_data_backup = raw_data.copy()

### Filter All-Star game
We see the All-Star game is included, we want to discard it as it is not a regular game:

In [3]:
def get_teams(games_stats: pd.DataFrame) -> List[str]:
    """
    Given a game statistics dataframe, returns the unique list of teams.
    Take both home + away teams if the algorithm is run early in the season
    """
    return pd.concat([raw_data.home_team, raw_data.away_team], ignore_index=True).unique().tolist()

In [4]:
teams = get_teams(raw_data)
print(teams)
filtered_teams = ['All Star France', 'All Star Monde']
raw_data = raw_data[~raw_data.home_team.isin(filtered_teams)]
teams = get_teams(raw_data)
print(teams)

# Validation
assert all(team not in raw_data['home_team'] and team not in raw_data['away_team'] for team in filtered_teams)
assert all(team not in teams for team in filtered_teams)

['Dijon', 'Châlons-Reims', 'Boulogne-Levallois', 'Monaco', 'Chalon/Saône', 'Boulazac', 'Cholet', 'Bourg-en-Bresse', 'Lyon-Villeurbanne', 'Roanne', 'Pau-Lacq-Orthez', 'Limoges', 'Le Mans', 'Strasbourg', 'Nanterre', 'Le Portel', 'Gravelines-Dunkerque', 'Orléans', 'All Star France', 'All Star Monde']
['Dijon', 'Châlons-Reims', 'Boulogne-Levallois', 'Monaco', 'Chalon/Saône', 'Boulazac', 'Cholet', 'Bourg-en-Bresse', 'Lyon-Villeurbanne', 'Roanne', 'Pau-Lacq-Orthez', 'Limoges', 'Le Mans', 'Strasbourg', 'Nanterre', 'Le Portel', 'Gravelines-Dunkerque', 'Orléans']


### Win/Loss ratio computation

Compute team wins/loss before each game:

In [5]:
teams_WL_tmp = pd.DataFrame(data=np.zeros((len(teams), 2)), dtype=np.int64, columns=['wins', 'losses'], index=teams) # Creating a temporary dataframe to hold current team win/loss
raw_data.sort_values(by='date', axis='index', ascending=True, inplace=True) # Sorting by ascending dates
raw_data['home_team_wins'] = 0
raw_data['home_team_losses'] = 0
raw_data['away_team_wins'] = 0
raw_data['away_team_losses'] = 0

In [6]:
for index, row in raw_data.iterrows():
    raw_data.at[index, 'home_team_wins'] = teams_WL_tmp.loc[row["home_team"]]['wins']
    raw_data.at[index, 'away_team_wins'] = teams_WL_tmp.loc[row["away_team"]]['wins']
    raw_data.at[index, 'home_team_losses'] = teams_WL_tmp.loc[row["home_team"]]['losses']
    raw_data.at[index, 'away_team_losses'] = teams_WL_tmp.loc[row["away_team"]]['losses']
    
    if row['home_score'] > row['away_score']:
        teams_WL_tmp.at[row["home_team"], 'wins'] = teams_WL_tmp.at[row["home_team"], 'wins'] + 1
        teams_WL_tmp.at[row["away_team"], 'losses'] = teams_WL_tmp.at[row["away_team"], 'losses'] + 1
    elif row['home_score'] < row['away_score']:
        teams_WL_tmp.at[row["away_team"], 'wins'] = teams_WL_tmp.at[row["away_team"], 'wins'] + 1
        teams_WL_tmp.at[row["home_team"], 'losses'] = teams_WL_tmp.at[row["home_team"], 'losses'] + 1

# Validation
display(raw_data.sample(n=5, random_state=42))

,date,home_team,home_score,away_team,away_score,home_qt_1_pts,away_qt_1_pts,home_qt_2_pts,away_qt_2_pts,home_qt_3_pts,...,away_ftm,away_blk,away_stl,away_tov,away_pf,away_pfd,home_team_wins,home_team_losses,away_team_wins,away_team_losses
220,2020-02-29,Nanterre,85,Boulazac,78,23,16,22,25,15,...,27,2,9,15,22,22,12,12,7,16
66,2019-11-08,Le Portel,98,Boulogne-Levallois,79,32,24,24,7,18,...,18,4,14,9,23,20,1,7,8,0
9,2019-09-25,Lyon-Villeurbanne,78,Boulazac,69,18,19,23,13,18,...,18,5,3,14,21,21,1,0,1,0
171,2020-01-19,Lyon-Villeurbanne,104,Bourg-en-Bresse,71,24,30,36,10,23,...,25,1,6,19,34,26,16,2,12,6
13,2019-09-28,Limoges,66,Le Portel,65,23,27,18,15,14,...,11,4,7,13,24,21,0,1,0,1


### Possessions & Pace

Computation of the number of possessions. We used formula $FGA + 0.44*FTA - ORB + TOV$. See notes for more in-depth information. We can then infer the pace (possessions per 40 minutes), by computing $\dfrac{40 * possessions}{minutes}$ to see how fast a team plays.  
(*Note that `minutes` column is divided by 5, as it is the total played, by all 5 players on the floor.*).

In [7]:
raw_data['home_possessions'] = round(raw_data['home_2pa'] + raw_data['home_3pa'] + 0.44 * raw_data['home_fta'] - raw_data['home_orbd'] + raw_data['home_tov'], 2)
raw_data['away_possessions'] = round(raw_data['away_2pa'] + raw_data['away_3pa'] + 0.44 * raw_data['away_fta'] - raw_data['away_orbd'] + raw_data['away_tov'], 2)
raw_data['home_pace'] = round((raw_data['home_possessions']*40)/(raw_data['minutes']/5), 2)
raw_data['away_pace'] = round((raw_data['away_possessions']*40)/(raw_data['minutes']/5), 2)

assert all(np.greater(raw_data[raw_data.minutes > 200].home_possessions, raw_data[raw_data.minutes > 200].home_pace))
assert all(np.equal(raw_data[raw_data.minutes == 200].home_pace, raw_data[raw_data.minutes == 200].home_possessions))
assert all(np.greater(raw_data[raw_data.minutes > 200].away_possessions, raw_data[raw_data.minutes > 200].away_pace))
assert all(np.equal(raw_data[raw_data.minutes == 200].away_pace, raw_data[raw_data.minutes == 200].away_possessions))

display(raw_data[['home_team', 'away_team', 'home_possessions', 'away_possessions', 'home_pace', 'away_pace']].sample(n=5, random_state=42))

,home_team,away_team,home_possessions,away_possessions,home_pace,away_pace
220,Nanterre,Boulazac,75.68,73.20,75.68,73.20
66,Le Portel,Boulogne-Levallois,83.44,84.00,83.44,84.00
9,Lyon-Villeurbanne,Boulazac,72.84,70.32,72.84,70.32
171,Lyon-Villeurbanne,Bourg-en-Bresse,72.28,73.96,72.28,73.96
13,Limoges,Le Portel,75.00,73.28,75.00,73.28


### Offensive Rating (ORtg), Defensive Rating (DRtg) and Net rating (NRtg)

Now that we have the pace, we can derive Offensive Rating, Defensive Rating and Net Rating. Offensive Rating is points scored per 100 possessions:  
$\dfrac{Pts * 100}{Poss}$  
  
Defensive rating is basically the opponent's offensive rating, and finally Net Rating is:  
$NRtg = ORtg - DRtg$.

In [8]:
raw_data['home_ortg'] = raw_data['home_score'] * 100 / raw_data['home_possessions']
raw_data['away_ortg'] = raw_data['away_score'] * 100 / raw_data['away_possessions']
raw_data['home_drtg'] = raw_data['away_ortg']
raw_data['away_drtg'] = raw_data['home_ortg']
raw_data['home_nrtg'] = raw_data['home_ortg'] - raw_data['home_drtg']
raw_data['away_nrtg'] = raw_data['away_ortg'] - raw_data['away_drtg']

display(raw_data[[
    'home_team', 
    'away_team', 
    'home_score', 
    'away_score', 
    'home_possessions', 
    'away_possessions',
    'home_ortg',
    'away_ortg',
    'home_drtg',
    'away_drtg',
    'home_nrtg',
    'away_nrtg',
]].sample(n=5, random_state=42))

,home_team,away_team,home_score,away_score,home_possessions,away_possessions,home_ortg,away_ortg,home_drtg,away_drtg,home_nrtg,away_nrtg
220,Nanterre,Boulazac,85,78,75.68,73.20,112.315011,106.557377,106.557377,112.315011,5.757634,-5.757634
66,Le Portel,Boulogne-Levallois,98,79,83.44,84.00,117.449664,94.047619,94.047619,117.449664,23.402045,-23.402045
9,Lyon-Villeurbanne,Boulazac,78,69,72.84,70.32,107.084020,98.122867,98.122867,107.084020,8.961153,-8.961153
171,Lyon-Villeurbanne,Bourg-en-Bresse,104,71,72.28,73.96,143.884892,95.997837,95.997837,143.884892,47.887055,-47.887055
13,Limoges,Le Portel,66,65,75.00,73.28,88.000000,88.700873,88.700873,88.000000,-0.700873,0.700873


### Compute average team statistics before each game

Pseudo code of the function to compute a stat for all previous game efficiently:
```
acc_dataframe <- Create dataframe [team=list of unique teams, sum_stats=np.Nan, nb_games_played=0]
raw_data <- Sort raw_data by game date

for game in raw_data:
    if first game of team:
        game[home_stat] <- NaN
    else:
        game[home_stat] <- acc_dataframe[game[home_team], sum_stats] / acc_dataframe[game[home_team], nb_games_played]
        
    acc_dataframe[team][sum_statistic] += game[home_stat]
    acc_dataframe[team][nb_game_played] += 1
```

In [9]:
def avg_stat(games_stats: pd.DataFrame, statistic: str):
    """
    Given a game statistics dataframe and a statistic category (rbd, blk, ast, etc.) 
    computes the average for each team (home, away) before each game.
    """
    teams = get_teams(games_stats)
    acc_dataframe = pd.DataFrame(data=np.zeros((len(teams), 2)), columns=['sum_' + statistic, 'games_played'], index=teams)
    raw_data.sort_values(by='date', axis='index', ascending=True, inplace=True) # Sorting by ascending dates
    
    for idx, game in games_stats.iterrows():
        for team in ['home', 'away']:
            # Computing average for the team
            if acc_dataframe.loc[game[team + '_team'], 'games_played'] == 0: # First game
                games_stats.loc[idx, team + '_avg_' + statistic] = np.nan
            else: # All other games
                games_stats.loc[idx, team + '_avg_' + statistic] = acc_dataframe.loc[game[team + '_team'], 'sum_' + statistic] / acc_dataframe.loc[game[team + '_team'], 'games_played']
    
            # Adding current game stats to the team accumulated stats.
            acc_dataframe.loc[game[team + '_team'], 'sum_' + statistic] += games_stats.loc[idx, team + '_' + statistic]
            acc_dataframe.loc[game[team + '_team'], 'games_played'] += 1

In [10]:
raw_data.columns

Index(['date', 'home_team', 'home_score', 'away_team', 'away_score',
       'home_qt_1_pts', 'away_qt_1_pts', 'home_qt_2_pts', 'away_qt_2_pts',
       'home_qt_3_pts', 'away_qt_3_pts', 'home_qt_4_pts', 'away_qt_4_pts',
       'home_ot_1_pts', 'away_ot_1_pts', 'home_ot_2_pts', 'away_ot_2_pts',
       'home_ot_3_pts', 'away_ot_3_pts', 'boxscore_url', 'minutes', 'home_ast',
       'home_drbd', 'home_orbd', 'home_2pa', 'home_2pm', 'home_3pa',
       'home_3pm', 'home_fta', 'home_ftm', 'home_blk', 'home_stl', 'home_tov',
       'home_pf', 'home_pfd', 'away_ast', 'away_drbd', 'away_orbd', 'away_2pa',
       'away_2pm', 'away_3pa', 'away_3pm', 'away_fta', 'away_ftm', 'away_blk',
       'away_stl', 'away_tov', 'away_pf', 'away_pfd', 'home_team_wins',
       'home_team_losses', 'away_team_wins', 'away_team_losses',
       'home_possessions', 'away_possessions', 'home_pace', 'away_pace',
       'home_ortg', 'away_ortg', 'home_drtg', 'away_drtg', 'home_nrtg',
       'away_nrtg'],
      dtype='obj

In [11]:
avg_stat(raw_data, 'score')
raw_data = raw_data.rename({'home_avg_score': 'home_avg_pts', 'away_avg_score': 'away_avg_pts'})
avg_stat(raw_data, 'qt_1_pts')
avg_stat(raw_data, 'qt_2_pts')
avg_stat(raw_data, 'qt_3_pts')
avg_stat(raw_data, 'qt_4_pts')
avg_stat(raw_data, 'ast')
avg_stat(raw_data, 'drbd')
avg_stat(raw_data, 'orbd')
avg_stat(raw_data, '2pm')
avg_stat(raw_data, '2pa')
avg_stat(raw_data, '3pm')
avg_stat(raw_data, '3pa')
avg_stat(raw_data, 'ftm')
avg_stat(raw_data, 'fta')
avg_stat(raw_data, 'blk')
avg_stat(raw_data, 'stl')
avg_stat(raw_data, 'tov')
avg_stat(raw_data, 'pf')
avg_stat(raw_data, 'pfd')
avg_stat(raw_data, 'possessions')
avg_stat(raw_data, 'pace')
avg_stat(raw_data, 'ortg')
avg_stat(raw_data, 'drtg')
avg_stat(raw_data, 'nrtg')
display(raw_data.iloc[random.randint(0, raw_data.shape[0])])

date             2020-01-17 00:00:00
home_team              Châlons-Reims
home_score                        78
away_team                    Le Mans
away_score                        87
                        ...         
away_avg_ortg                 112.65
home_avg_drtg                111.353
away_avg_drtg                114.541
home_avg_nrtg                1.00684
away_avg_nrtg               -1.89176
Name: 164, Length: 111, dtype: object

### Saving processed data

In [12]:
raw_data.to_csv('data/processed_data.csv', index=False)